# Data and Constants

In [15]:
import pandas as pd
import numpy as np
import re
from scipy.stats import stats
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from scipy.stats import norm
import matplotlib.pyplot as plt
from pycaret.regression import *
from multiprocessing import Pool, cpu_count
from concurrent.futures import ThreadPoolExecutor

In [16]:
lipid_path = 'data/section12/lipids_section_12.h5'
gene_path = 'data/section12/genes_section_12.h5'

# Helper Functions

In [17]:
def gaussian_weight(distance, std_dev):
    return norm.pdf(distance, 0, std_dev)

In [18]:
def exponential_decay(distance, threshold, decay_rate):
    adjusted_distance = distance - threshold
    return np.exp(-decay_rate * adjusted_distance)

In [19]:

def filter_dataframe_from_file(df, filepath):
    # Read the text file and store the entries in a list
    with open(filepath, 'r') as file:
        entries = file.read().splitlines()

    # Filter the DataFrame based on the index matching the entries
    filtered_df = df.loc[df.index.isin(entries)]

    return filtered_df

# Loading Dataset

In [20]:
# Loading the dataset
lipids_section_12 = pd.read_hdf(lipid_path)
genes_section_12 = pd.read_hdf(gene_path)

# Need to remove the trailing naming scheme added before
new_column_names = [re.sub(r'_(\d+)$', '', col) for col in lipids_section_12.columns]
lipids_section_12.columns = new_column_names

# Create cKDTree for fast query of neighbors

In [21]:
# Create a KDTree object for the genes
genes_coords = genes_section_12[['y_ccf', 'z_ccf']].values
genes_kdtree = cKDTree(genes_coords)

# Extract coordinates for lipids
lipids_coords = lipids_section_12[['y_ccf', 'z_ccf']].values

# Nearest Neighbor selection

In [7]:
# Find the indices of the closest gene for each lipid point
_, indices = genes_kdtree.query(lipids_coords, k=1)

# Initialize an empty array for aggregated gene data
aggregated_gene_data = np.zeros((len(lipids_coords), genes_section_12.iloc[:, 46:-50].shape[1]))

# Aggregate gene data based on the closest neighbor
for i, gene_index in enumerate(indices):
    aggregated_gene_data[i] = genes_section_12.iloc[gene_index, 46:-50]

# Convert the aggregated data into a DataFrame
aggregated_gene_data_df = pd.DataFrame(aggregated_gene_data, columns=genes_section_12.columns[46:-50])

# Average of nearest neighbors

In [44]:
# Query to get the 5000 closest genes for each lipid point
_, indices = genes_kdtree.query(lipids_coords, k=10000)

# Pre-allocate memory for aggregated gene data
n_genes = genes_section_12.iloc[:, 46:-50].shape[1]
aggregated_gene_data = np.zeros((len(lipids_coords), n_genes))

# Define a function to aggregate data
def aggregate_data(i):
    gene_indices = indices[i]
    data = genes_section_12.iloc[gene_indices, 46:-50].values
    return data.mean(axis=0)

# Parallel processing
with Pool(cpu_count()) as pool:
    results = pool.map(aggregate_data, range(len(lipids_coords)))

# Assign the results back to the array
for i, res in enumerate(results):
    aggregated_gene_data[i] = res

# Convert the aggregated data into a DataFrame
aggregated_gene_data_df = pd.DataFrame(aggregated_gene_data, columns=genes_section_12.columns[46:-50])

# Exponential Decay

In [14]:
# Find the distances and indices of the 6 closest genes for each lipid point
distances, indices = genes_kdtree.query(lipids_coords, k=6)

# Calculate the average distance
average_closest_distance = np.mean(distances)

# Query to get the 100 closest genes and their distances
distances, indices = genes_kdtree.query(lipids_coords, k=1000)

# Initialize an empty array for aggregated gene data
aggregated_gene_data = np.zeros((len(lipids_coords), genes_section_12.iloc[:, 46:-50].shape[1]))

# Perform weighted aggregation
for i, (gene_indices, dists) in enumerate(zip(indices, distances)):
    # Weights based on distance, with a penalty for distances greater than the average closest distance
    weights = np.where(dists <= average_closest_distance, 1, exponential_decay(dists, average_closest_distance, 3))  # Apply penalty for dist > average_closest_distance
    weighted_data = genes_section_12.iloc[gene_indices, 46:-50] * weights[:, np.newaxis]
    aggregated_gene_data[i] = weighted_data.sum(axis=0) / weights.sum() if weights.sum() > 0 else np.zeros(genes_section_12.iloc[:, 46:-50].shape[1])

# Convert the aggregated data into a DataFrame
aggregated_gene_data_df = pd.DataFrame(aggregated_gene_data, columns=genes_section_12.columns[46:-50])


# K neighbors gaussian mean of genes for a given lipids datapoint

In [8]:
# Find the distances and indices of the closest 2500 genes for each lipid point
distances, _ = genes_kdtree.query(lipids_coords, k=50)

# Calculate the std of the distances
std_closest_distance = np.std(distances, axis=1)

# Query to get the 5000 closest genes and their distances
distances, indices = genes_kdtree.query(lipids_coords, k=100)

# Pre-allocate memory for aggregated gene data
n_genes = genes_section_12.iloc[:, 46:-50].shape[1]
aggregated_gene_data = np.zeros((len(lipids_coords), n_genes))

# Define a function to aggregate data
def aggregate_data(i):
    dists = distances[i]
    gene_indices = indices[i]
    weights = gaussian_weight(dists, std_closest_distance[i])
    weighted_data = genes_section_12.iloc[gene_indices, 46:-50].values * weights[:, np.newaxis]
    return weighted_data.sum(axis=0) / weights.sum() if weights.sum() > 0 else np.zeros(n_genes)

# Parallel processing
with Pool(cpu_count()) as pool:
    results = pool.map(aggregate_data, range(len(lipids_coords)))

# Assign the results back to the array
for i, res in enumerate(results):
    aggregated_gene_data[i] = res

# Convert the aggregated data into a DataFrame
aggregated_gene_data_df = pd.DataFrame(aggregated_gene_data, columns=genes_section_12.columns[46:-50])

# Take all the points into consideration

In [8]:
def process_lipid_point(lipid_coord, genes_coords, gene_data, std_closest_distance):
    """Process a single lipid point and return the aggregated data."""
    # Calculate distances to all gene points
    dists = np.linalg.norm(genes_coords - lipid_coord, axis=1)

    # Weights based on distance, with a penalty for distances greater than the average closest distance
    weights = gaussian_weight(dists, std_closest_distance)

    # Perform weighted aggregation
    weighted_data = gene_data * weights[:, np.newaxis]
    return weighted_data.sum(axis=0) / weights.sum() if weights.sum() > 0 else np.zeros(gene_data.shape[1])

In [9]:

gene_data_columns = genes_section_12.columns[46:-50]
gene_data_shape = genes_section_12[gene_data_columns].shape[1]

# Initialize an empty array for aggregated gene data
aggregated_gene_data = np.zeros((len(lipids_coords), gene_data_shape))

distances, _ = genes_kdtree.query(lipids_coords, k=6)
std_closest_distance = np.std(distances)

# Extract gene data only once
gene_data = genes_section_12[gene_data_columns].values

# Parallel processing
with Pool(cpu_count()) as pool:
    results = pool.starmap(process_lipid_point, [(lipid_coord, genes_coords, gene_data, std_closest_distance) for lipid_coord in lipids_coords])

# Combine results into aggregated_gene_data
for i, result in enumerate(results):
    aggregated_gene_data[i] = result

# Convert the aggregated data into a DataFrame
aggregated_gene_data_df = pd.DataFrame(aggregated_gene_data, columns=gene_data_columns)


# Resulting genes and lipids DataFrames

In [45]:
aggregated_gene_data_df


,ENSMUST00000028118,ENSMUST00000028280,ENSMUST00000030676,ENSMUST00000047328,ENSMUST00000057021,ENSMUST00000090697,ENSMUST00000091554,ENSMUST00000162772,ENSMUST00000021284,ENSMUST00000022195,...,ENSMUST00000109964,ENSMUST00000114553,ENSMUST00000152412,ENSMUST00000159365,ENSMUST00000175965,ENSMUST00000196378,ENSMUST00000228095,ENSMUST00000000219,ENSMUST00000035577,ENSMUST00000060943
0,0.198063,0.405779,0.645885,0.383614,0.121224,0.282417,0.202944,0.350521,0.165358,0.048809,...,0.043493,0.022140,0.050515,0.255254,0.134016,0.232900,0.021044,0.045652,0.053825,0.071600
1,0.199219,0.407315,0.647939,0.381005,0.120874,0.283876,0.203127,0.350157,0.166456,0.048699,...,0.043711,0.022134,0.050674,0.255004,0.134805,0.233086,0.020933,0.045395,0.054033,0.071429
2,0.200234,0.409451,0.648729,0.378686,0.121325,0.285415,0.202980,0.349893,0.166421,0.048739,...,0.043714,0.022311,0.050970,0.254824,0.135325,0.232859,0.021101,0.045156,0.054982,0.070884
3,0.200904,0.409934,0.650612,0.375842,0.121098,0.286163,0.202869,0.350290,0.166803,0.048958,...,0.043470,0.022196,0.051176,0.254139,0.135281,0.232807,0.021087,0.045124,0.055081,0.070209
4,0.200839,0.410936,0.650788,0.372453,0.121199,0.286149,0.202056,0.349187,0.167166,0.049106,...,0.043459,0.022336,0.051197,0.254119,0.135473,0.232933,0.021023,0.045203,0.056045,0.070500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94742,0.220339,0.331264,0.419088,0.408906,0.105012,0.158043,0.229835,0.502113,0.201653,0.066926,...,0.031479,0.085143,0.035111,0.134605,0.090077,0.165250,0.017055,0.042562,0.079678,0.015618
94743,0.220853,0.328946,0.419589,0.404937,0.104671,0.159784,0.230158,0.498829,0.203246,0.067099,...,0.031592,0.086388,0.034953,0.134294,0.091420,0.164871,0.016993,0.044765,0.078859,0.016414
94744,0.221795,0.327225,0.418933,0.400377,0.104988,0.163358,0.231359,0.495328,0.205348,0.067729,...,0.031874,0.088803,0.034455,0.134145,0.093561,0.164224,0.016992,0.046617,0.078584,0.016811
94745,0.222671,0.326264,0.417704,0.398694,0.104390,0.165506,0.232134,0.492282,0.206897,0.067477,...,0.031719,0.090337,0.033959,0.133361,0.094879,0.162158,0.017215,0.048513,0.078311,0.017056


In [46]:
section_12_lipids_only = lipids_section_12.iloc[:, 3:-3]
section_12_lipids_only

,LPC O-16:2,LPC 16:0_dup,LPC O- 18:3,LPC O-18:2,LPC O-16:2_dup,LPC 15:1,LPC 18:1,LPC 18:0_dup,LPC 16:0,LPC O-18:3,...,SM(t42:1),PC(40:7),PC 40:6_dup,PG(42:6),Hex2Cer 32:0,SHexCer 38:1;3,PE(44:11(OH)),PC(40:4),PS(40:4),PIP(O-36:5)
section12_pixel23_121,0.000140,0.000112,0.000116,0.000125,0.000214,0.000100,0.0001,0.000197,0.000179,0.0001,...,0.0001,0.000100,0.000241,0.000179,0.0001,0.000100,0.0001,0.000261,0.0001,0.000360
section12_pixel23_122,0.000213,0.000112,0.000114,0.000125,0.000204,0.000162,0.0001,0.000100,0.000181,0.0001,...,0.0001,0.000114,0.000395,0.000208,0.0001,0.000316,0.0001,0.000268,0.0001,0.000100
section12_pixel23_123,0.000154,0.000100,0.000117,0.000134,0.000195,0.000151,0.0001,0.000232,0.000179,0.0001,...,0.0001,0.000114,0.000233,0.000203,0.0001,0.000100,0.0001,0.000232,0.0001,0.000100
section12_pixel23_124,0.000147,0.000113,0.000114,0.000136,0.000229,0.000154,0.0001,0.000100,0.000120,0.0001,...,0.0001,0.000114,0.000285,0.000187,0.0001,0.000255,0.0001,0.000100,0.0001,0.000366
section12_pixel23_125,0.000229,0.000112,0.000115,0.000206,0.000100,0.000100,0.0001,0.000100,0.000122,0.0001,...,0.0001,0.000100,0.000247,0.000179,0.0001,0.000323,0.0001,0.000100,0.0001,0.000100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
section12_pixel308_157,0.000139,0.000100,0.000100,0.000100,0.000100,0.000100,0.0001,0.000100,0.000100,0.0001,...,0.0001,0.000100,0.000100,0.000100,0.0001,0.000100,0.0001,0.000100,0.0001,0.000100
section12_pixel308_158,0.000139,0.000100,0.000100,0.000100,0.000100,0.000100,0.0001,0.000100,0.000119,0.0001,...,0.0001,0.000100,0.000100,0.000100,0.0001,0.000100,0.0001,0.000100,0.0001,0.000100
section12_pixel308_159,0.000155,0.000112,0.000117,0.000100,0.000100,0.000100,0.0001,0.000100,0.000100,0.0001,...,0.0001,0.000100,0.000100,0.000100,0.0001,0.000100,0.0001,0.000100,0.0001,0.000100
section12_pixel308_160,0.000141,0.000113,0.000100,0.000100,0.000100,0.000100,0.0001,0.000237,0.000119,0.0001,...,0.0001,0.000100,0.000100,0.000100,0.0001,0.000100,0.0001,0.000100,0.0001,0.000100


In [47]:
aggregated_gene_data_df = aggregated_gene_data_df.reset_index(drop=True)
section_12_lipids_only = section_12_lipids_only.reset_index(drop=True)

# Simplify lipid names

In [48]:
def rename_lipid(lipid_name):
    # Example renaming scheme - customize as needed
    new_name = lipid_name.replace(':', ';').replace('\xa0', ' ')
    return new_name

section_12_lipids_only.columns = [rename_lipid(lipid) for lipid in section_12_lipids_only.columns] 

# Selecting best model and training it and see the results

In [ ]:
# Loop through each lipid type based on the lipid names in the column headers.
for i, lipid_name in enumerate(section_12_lipids_only.columns):
    
    # Create a DataFrame to store genes and the corresponding lipid values
    # Use the aggregated gene data (aggregated_gene_data_df) and the lipid data
    df = aggregated_gene_data_df.copy()
    
    # Add a column for the lipid values
    # The index should match the index of the aggregated gene data
    df[lipid_name] = section_12_lipids_only.iloc[:, i]
    
    # Step 1: Setup the environment in PyCaret
    setup(data=df, fold=5, session_id=42, use_gpu=True, verbose=False, preprocess=False)
    
    # Step 2: Select best model
    #best_model = compare_models(fold=5, exclude=['gbr', 'ada'])

    # Step 2: Create the model  
    model = create_model('catboost')
    
    plot_model(model, plot='feature')

    # Step 3: Finalize the model
    #final_model = finalize_model(model)
    
    # Step 4: Save the model
    #save_model(model, f'models/{lipid_name}_model_{i}')

# Evaluating One Model using K-Cross Validation

In [49]:
results_df = pd.DataFrame(columns=['Lipid', 'R2_mean', 'MAPE_mean'])

for i, lipid_name in enumerate(section_12_lipids_only.columns):
    df = aggregated_gene_data_df.copy()
    df[lipid_name] = section_12_lipids_only.iloc[:, i]
    
    s = setup(data=df, target=lipid_name, fold=5, session_id=42, verbose=False, use_gpu=True, preprocess=False)
    
    model = create_model('catboost', verbose=False)
    
    #fine_tuned = tune_model(model, search_library='optuna',optimize="MAPE", n_iter=100, early_stopping=True)
    
    # Retrieving cross-validation results
    metrics = pull()
    
    r2_mean = metrics.loc['Mean','R2']
    mape_mean = metrics.loc['Mean', 'MAPE']
    
    print("R2 mean: ", r2_mean)
    print("MAPE mean: ", mape_mean)
    
    # Append the results to the DataFrame
    results_df = results_df.append({'Lipid': lipid_name, 'R2_mean': r2_mean, 'MAPE_mean': mape_mean}, ignore_index=True)

results_df


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

...

[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


R2 mean:  0.1996
MAPE mean:  0.8599


,Lipid,R2_mean,MAPE_mean
0,LPC O-16;2,0.6194,0.2748
1,LPC 16;0_dup,0.2235,0.3306
2,LPC O- 18;3,0.4008,0.2719
3,LPC O-18;2,0.3570,0.2515
4,LPC O-16;2_dup,0.5520,0.2585
...,...,...,...
197,SHexCer 38;1;3,0.4261,0.4244
198,PE(44;11(OH)),0.0969,0.9345
199,PC(40;4),0.5270,0.3003
200,PS(40;4),0.1369,0.9423


In [50]:
results_df.to_csv('results.csv')